In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
df=pd.read_csv('hwdata.csv')


In [12]:
#select columns
df=df[['latitude','longitude','price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']]
df.head()


,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64749,-73.97237,149,1,9,0.21,6,365
1,40.75362,-73.98377,225,1,45,0.38,2,355
2,40.80902,-73.94190,150,3,0,NaN,1,365
3,40.68514,-73.95976,89,1,270,4.64,1,194
4,40.79851,-73.94399,80,10,9,0.10,1,0


Question 1

In [14]:
df.isnull().sum()
#feature:reviews_per_month  
#na's:10052

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Question 2

In [17]:
minig=df.minimum_nights.values
np.median(minig)

3.0

Spliting the data

In [18]:
n=len(df)
n_val=int(len(df)*0.2)
n_test=int(len(df)*0.2)
n_train=n -n_val -n_test

idx=np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
idx

array([  879, 44383, 15394, ..., 38158,   860, 15795])

In [19]:
df_train=df.iloc[idx[:n_train]]
df_val=df.iloc[idx[n_train:n_val+n_train]]
df_test=df.iloc[idx[n_train+n_val:]]
df_train.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
879,40.64354,-73.97777,89,3,62,0.71,1,189
44383,40.70666,-73.90779,30,21,0,NaN,1,73
15394,40.76116,-73.99016,120,2,17,0.43,1,0
43230,40.70763,-74.01050,470,2,5,1.88,327,272
16332,40.79658,-73.93287,199,2,30,0.80,1,30


In [20]:
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_train.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64354,-73.97777,89,3,62,0.71,1,189
1,40.70666,-73.90779,30,21,0,NaN,1,73
2,40.76116,-73.99016,120,2,17,0.43,1,0
3,40.70763,-74.01050,470,2,5,1.88,327,272
4,40.79658,-73.93287,199,2,30,0.80,1,30


In [21]:
y_train=np.log1p(df_train.price.values)
y_val=np.log1p(df_val.price.values)
y_test=np.log1p(df_test.price.values)
y_train

array([4.49980967, 3.4339872 , 4.79579055, ..., 4.61512052, 4.2341065 ,
       3.8286414 ])

In [22]:
del df_train['price']
del df_test['price']
del df_val['price']

In [23]:
df_train.head()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64354,-73.97777,3,62,0.71,1,189
1,40.70666,-73.90779,21,0,NaN,1,73
2,40.76116,-73.99016,2,17,0.43,1,0
3,40.70763,-74.01050,2,5,1.88,327,272
4,40.79658,-73.93287,2,30,0.80,1,30


Question 3

In [28]:
def train_linear_regression(x,y):
    ones=np.ones(x.shape[0])
    x=np.column_stack([ones,x])
    XTX=x.T.dot(x)
    XTX_inv=np.linalg.inv(XTX)
    w_full=XTX_inv.dot(x.T).dot(y)
    return w_full[0],w_full[1:]
def rmse(y,y_pred):
    error=y-y_pred
    se=error**2
    mse=se.mean()
    return np.sqrt(mse)

Filling with 0

In [40]:
X_train=df_train.fillna(0).values
w0,w=train_linear_regression(X_train,y_train)
y_pred=X_train.dot(w)+w0
round(rmse(y_pred,y_train),2)

0.64

Filling with mean

In [38]:
reviews=df_train['reviews_per_month'].values
new_reviews=reviews[np.logical_not(np.isnan(reviews))]
mean_reviews=np.mean(new_reviews)
mean_reviews

1.3685492094776983

In [43]:
X_train=df_train.fillna(mean_reviews).values
w0,w=train_linear_regression(X_train,y_train)
y_pred=X_train.dot(w)+w0
round(rmse(y_pred,y_train),2)

0.64

Question 4

In [44]:
def train_linear_regression_reg(x,y,r):
    ones=np.ones(x.shape[0])
    x=np.column_stack([ones,x])
    
    XTX=x.T.dot(x)
    XTX= XTX +r*np.eye(XTX.shape[0])
    
    XTX_inv=np.linalg.inv(XTX)
    w_full=XTX_inv.dot(x.T).dot(y)
    return w_full[0],w_full[1:]

In [46]:
def prepare_X(df):
    df=df.copy()
    df_num=df
    df_num=df_num.fillna(0)
    X=df_num.values
    return X

In [49]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:

    X_train=prepare_X(df_train)
    w0,w=train_linear_regression_reg(X_train,y_train,r=r)#train


    x_val=prepare_X(df_val)#val
    y_pred= w0+ np.dot(x_val,w)
    score=rmse(y_val,y_pred)  #temos de encontrar o melhor valor para r
    
    print(r,round(score,4))

0 0.643
1e-06 0.643
0.0001 0.6431
0.001 0.6438
0.01 0.6558
0.1 0.6773
1 0.6823
5 0.6828
10 0.6828


Question 5

In [50]:
n=len(df)
n_val=int(len(df)*0.2)
n_test=int(len(df)*0.2)
n_train=n -n_val -n_test

idx=np.arange(n)


In [59]:
seed=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
def separate_data(seednumb):
    np.random.seed(seednumb)
    np.random.shuffle(idx)
    idx

    df_train=df.iloc[idx[:n_train]]
    df_val=df.iloc[idx[n_train:n_val+n_train]]
    df_test=df.iloc[idx[n_train+n_val:]]

    df_train=df_train.reset_index(drop=True)
    df_val=df_val.reset_index(drop=True)
    df_test=df_test.reset_index(drop=True)

    y_train=np.log1p(df_train.price.values)
    y_val=np.log1p(df_val.price.values)
    y_test=np.log1p(df_test.price.values)
    return df_train,df_val,y_train,y_val

In [64]:
def train_linear_regression(x,y):
    ones=np.ones(x.shape[0])
    x=np.column_stack([ones,x])
    XTX=x.T.dot(x)
    XTX_inv=np.linalg.inv(XTX)
    w_full=XTX_inv.dot(x.T).dot(y)
    return w_full[0],w_full[1:]



In [65]:
score=[]
for i in range(len(seed)):
    df_train,df_val,y_train,y_val=separate_data(i)
    X_train=df_train.fillna(0).values
    w0,w=train_linear_regression(X_train,y_train)

    x_val=prepare_X(df_val)#val
    y_pred= w0+ np.dot(x_val,w)
    score.append(rmse(y_val,y_pred))

In [67]:
round(np.std(score),3)

0.015

Question 6

In [70]:
n=len(df)
n_val=int(len(df)*0.2)
n_test=int(len(df)*0.2)
n_train=n -n_val -n_test

idx=np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)
idx

df_train=df.iloc[idx[:n_train]]
df_val=df.iloc[idx[n_train:n_val+n_train]]
df_test=df.iloc[idx[n_train+n_val:]]

df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

y_train=np.log1p(df_train.price.values)
y_val=np.log1p(df_val.price.values)
y_test=np.log1p(df_test.price.values)

del df_train['price']
del df_test['price']
del df_val['price']

In [73]:
df_full_train=pd.concat([df_train,df_val])
df_full_train=df_full_train.reset_index(drop=True)
df_full_train=df_full_train.fillna(0)
X_full_train=prepare_X(df_full_train)
y_full_train=np.concatenate([y_train,y_val])

In [74]:
w0,w=train_linear_regression_reg(X_full_train,y_full_train,r=0.001)

In [76]:
#prepare test dataset
x_test=prepare_X(df_test)
y_pred= w0+ np.dot(x_test,w)
score=rmse(y_test,y_pred)
round(score,2) #resultado parecido ao anterior, nao funcinou por sorte

0.65